In [1]:
import numpy as np
import tensorflow as tf

initial = [np.random.random_sample(), np.random.random_sample()]

/Users/clint/virtualenv/tf/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


## First-order Minimization

In [2]:
sess = tf.InteractiveSession()

vs = tf.Variable(initial, dtype='float32')
f = vs[0]**2 + 2.0*vs[0]*vs[1] + 3.0*vs[1]**2 + 4.0*vs[0] + 5.0*vs[1] + 6.0
sess.run(tf.global_variables_initializer())

learning_rate = 0.2
for e in range(10):
    print('EPOCH', e, 'f=%.3f'%f.eval(), 'variables=', ['%.3f'%v for v in vs.eval()])
    
    grads = tf.gradients(f, vs)[0]
    delta = - grads*learning_rate
    print('\t \tgradients=', ['%.3f'%g for g in grads.eval()],
          'delta=', ['%.3f'%d for d in delta.eval()])
    op = vs.assign( vs + delta )
    op.eval()

print('FINAL  ', 'f=%.3f'%f.eval(), 'variables=', ['%.3f'%v for v in vs.eval()])
sess.close()

EPOCH 0 f=9.758 variables= ['0.364', '0.324']
	 	gradients= ['5.376', '7.671'] delta= ['-1.075', '-1.534']
EPOCH 1 f=3.726 variables= ['-0.711', '-1.210']
	 	gradients= ['0.157', '-3.685'] delta= ['-0.031', '0.737']
EPOCH 2 f=2.589 variables= ['-0.743', '-0.473']
	 	gradients= ['1.568', '0.674'] delta= ['-0.314', '-0.135']
EPOCH 3 f=2.244 variables= ['-1.056', '-0.608']
	 	gradients= ['0.671', '-0.762'] delta= ['-0.134', '0.152']
EPOCH 4 f=2.085 variables= ['-1.190', '-0.456']
	 	gradients= ['0.707', '-0.116'] delta= ['-0.141', '0.023']
EPOCH 5 f=1.997 variables= ['-1.332', '-0.433']
	 	gradients= ['0.471', '-0.260'] delta= ['-0.094', '0.052']
EPOCH 6 f=1.946 variables= ['-1.426', '-0.381']
	 	gradients= ['0.386', '-0.136'] delta= ['-0.077', '0.027']
EPOCH 7 f=1.917 variables= ['-1.503', '-0.353']
	 	gradients= ['0.286', '-0.127'] delta= ['-0.057', '0.025']
EPOCH 8 f=1.900 variables= ['-1.561', '-0.328']
	 	gradients= ['0.223', '-0.089'] delta= ['-0.045', '0.018']
EPOCH 9 f=1.889 varia

## Second-order Minimization

In [3]:
sess = tf.InteractiveSession()

vs = tf.Variable(initial)
f = vs[0]**2 + 2.0*vs[0]*vs[1] + 3.0*vs[1]**2 + 4.0*vs[0] + 5.0*vs[1] + 6.0
sess.run(tf.global_variables_initializer())

for e in range(3):
    print('EPOCH', e, 'f=%.3f'%f.eval(), 'variables=', ['%.3f'%v for v in vs.eval()])
    
    grads = tf.gradients(f, vs)[0]
    hessi = tf.hessians(f, vs)[0]
    hessi_inv = tf.matrix_inverse(hessi)
    delta = - tf.reshape( tf.matmul(hessi_inv, tf.reshape(grads, [-1, 1])), [2] )
    print('\t \tgradients=', ['%.3f'%g for g in grads.eval()],
          'hessi_inv=', ('%s'%hessi_inv.eval()).replace('\n', ','),
          'delta=', ['%.3f'%d for d in delta.eval()])

    op = vs.assign( vs + delta )
    op.eval()

print('FINAL  ', 'f=%.3f'%f.eval(), 'variables=', ['%.3f'%v for v in vs.eval()])
sess.close()

EPOCH 0 f=9.758 variables= ['0.364', '0.324']
	 	gradients= ['5.376', '7.671'] hessi_inv= [[ 0.75 -0.25], [-0.25  0.25]] delta= ['-2.114', '-0.574']
EPOCH 1 f=1.875 variables= ['-1.750', '-0.250']
	 	gradients= ['-0.000', '-0.000'] hessi_inv= [[ 0.75 -0.25], [-0.25  0.25]] delta= ['-0.000', '0.000']
EPOCH 2 f=1.875 variables= ['-1.750', '-0.250']
	 	gradients= ['0.000', '0.000'] hessi_inv= [[ 0.75 -0.25], [-0.25  0.25]] delta= ['0.000', '-0.000']
FINAL   f=1.875 variables= ['-1.750', '-0.250']


## Second-order Minimization self implemented Hessian

In [4]:
from functools import reduce
from tensorflow.python.ops import gradients_impl
from tensorflow.python.ops import array_ops, tensor_array_ops, control_flow_ops

def hessians_highrank(ys, xs, gradients=None, name="hessians", colocate_gradients_with_ops=False,
            gate_gradients=False, aggregation_method=None):
  """Constructs the Hessian (one or more rank matrix) of sum of `ys` with respect to `x` in `xs`.
  `hessians_highrank()` adds ops to the graph to output the Hessian matrix of `ys`
  with respect to `xs`.  It returns a list of `Tensor` of length `len(xs)`
  where each tensor is the Hessian of `sum(ys)`. This function currently
  only supports evaluating the Hessian with respect to (a list of) one-
  dimensional tensors.
  The Hessian is a matrix of second-order partial derivatives of a scalar
  tensor (see https://en.wikipedia.org/wiki/Hessian_matrix for more details).
  Args:
    ys: A `Tensor` or list of tensors to be differentiated.
    xs: A `Tensor` or list of tensors to be used for differentiation.
    name: Optional name to use for grouping all the gradient ops together.
      defaults to 'hessians'.
    colocate_gradients_with_ops: See `gradients()` documentation for details.
    gate_gradients: See `gradients()` documentation for details.
    aggregation_method: See `gradients()` documentation for details.
  Returns:
    A list of Hessian matrices of `sum(ys)` for each `x` in `xs`.
  Raises:
    LookupError: if one of the operations between `xs` and `ys` does not
      have a registered gradient function.
  """
  xs = gradients_impl._AsList(xs)
  kwargs = {
    'colocate_gradients_with_ops': colocate_gradients_with_ops,
    'gate_gradients': gate_gradients,
    'aggregation_method': aggregation_method
  }
  # Compute first-order derivatives and iterate for each x in xs.
  hessians = []
  _gradients = tf.gradients(ys, xs, **kwargs) if gradients is None else gradients
  for i, _gradient, x in zip(range(len(xs)), _gradients, xs):
    shape = x.shape
    _gradient = tf.reshape(_gradient, [-1])
    
    n = tf.size(x)
    loop_vars = [
      array_ops.constant(0, tf.int32),
      tensor_array_ops.TensorArray(x.dtype, n)
    ]
    _, hessian = control_flow_ops.while_loop(
      lambda j, _: j < n,
      lambda j, result: (j + 1, result.write(j, tf.gradients(_gradient[j], x, **kwargs)[0])),
      loop_vars
    )
    hessians.append(hessian.stack())
  return hessians

In [5]:
sess = tf.InteractiveSession()

vs = tf.Variable(initial)
f = vs[0]**2 + 2.0*vs[0]*vs[1] + 3.0*vs[1]**2 + 4.0*vs[0] + 5.0*vs[1] + 6.0
sess.run(tf.global_variables_initializer())

for e in range(3):
    print('EPOCH', e, 'f=%.3f'%f.eval(), 'variables=', ['%.3f'%v for v in vs.eval()])
    
    grads = tf.gradients(f, vs)[0]
    hessi = hessians_highrank(f, vs)[0]
    hessi_inv = tf.matrix_inverse(hessi)
    delta = - tf.reshape( tf.matmul(hessi_inv, tf.reshape(grads, [-1, 1])), [2] )
    print('\t \tgradients=', ['%.3f'%g for g in grads.eval()],
          'hessi_inv=', ('%s'%hessi_inv.eval()).replace('\n', ','),
          'delta=', ['%.3f'%d for d in delta.eval()])

    op = vs.assign( vs + delta )
    op.eval()

print('FINAL  ', 'f=%.3f'%f.eval(), 'variables=', ['%.3f'%v for v in vs.eval()])
sess.close()

EPOCH 0 f=9.758 variables= ['0.364', '0.324']
	 	gradients= ['5.376', '7.671'] hessi_inv= [[ 0.75 -0.25], [-0.25  0.25]] delta= ['-2.114', '-0.574']
EPOCH 1 f=1.875 variables= ['-1.750', '-0.250']
	 	gradients= ['-0.000', '-0.000'] hessi_inv= [[ 0.75 -0.25], [-0.25  0.25]] delta= ['-0.000', '0.000']
EPOCH 2 f=1.875 variables= ['-1.750', '-0.250']
	 	gradients= ['0.000', '0.000'] hessi_inv= [[ 0.75 -0.25], [-0.25  0.25]] delta= ['0.000', '-0.000']
FINAL   f=1.875 variables= ['-1.750', '-0.250']


## Second-order Minimization self implemented Diagonal Hessian

### brute force

In [6]:
sess = tf.InteractiveSession()

vs = tf.Variable(initial)
f = vs[0]**2 + 2.0*vs[0]*vs[1] + 3.0*vs[1]**2 + 4.0*vs[0] + 5.0*vs[1] + 6.0
sess.run(tf.global_variables_initializer())

for e in range(10):
    print('EPOCH', e, 'f=%.3f'%f.eval(), 'variables=', ['%.3f'%v for v in vs.eval()])
    
    grads = tf.gradients(f, vs, stop_gradients=vs)[0]
    hessi = hessians_highrank(f, vs)[0] * tf.eye(2)
    
    hessi_inv = tf.matrix_inverse(hessi)
    delta = - tf.reshape( tf.matmul(hessi_inv, tf.reshape(grads, [-1, 1])), [2] )
    print('\t \tgradients=', ['%.3f'%g for g in grads.eval()],
          'hessi_inv=', ('%s'%hessi_inv.eval()).replace('   ', '').replace('\n', ','),
          'delta=', ['%.3f'%d for d in delta.eval()])

    op = vs.assign( vs + delta )
    op.eval()

print('FINAL  ', 'f=%.3f'%f.eval(), 'variables=', ['%.3f'%v for v in vs.eval()])
sess.close()

EPOCH 0 f=9.758 variables= ['0.364', '0.324']
	 	gradients= ['5.376', '7.671'] hessi_inv= [[ 0.50.  ], [ 0. 0.16666667]] delta= ['-2.688', '-1.279']
EPOCH 1 f=4.503 variables= ['-2.324', '-0.955']
	 	gradients= ['-2.557', '-5.376'] hessi_inv= [[ 0.50.  ], [ 0. 0.16666667]] delta= ['1.279', '0.896']
EPOCH 2 f=2.751 variables= ['-1.045', '-0.059']
	 	gradients= ['1.792', '2.557'] hessi_inv= [[ 0.50.  ], [ 0. 0.16666667]] delta= ['-0.896', '-0.426']
EPOCH 3 f=2.167 variables= ['-1.941', '-0.485']
	 	gradients= ['-0.852', '-1.792'] hessi_inv= [[ 0.50.  ], [ 0. 0.16666667]] delta= ['0.426', '0.299']
EPOCH 4 f=1.972 variables= ['-1.515', '-0.186']
	 	gradients= ['0.597', '0.852'] hessi_inv= [[ 0.50.  ], [ 0. 0.16666667]] delta= ['-0.299', '-0.142']
EPOCH 5 f=1.907 variables= ['-1.814', '-0.328']
	 	gradients= ['-0.284', '-0.597'] hessi_inv= [[ 0.50.  ], [ 0. 0.16666667]] delta= ['0.142', '0.100']
EPOCH 6 f=1.886 variables= ['-1.672', '-0.229']
	 	gradients= ['0.199', '0.284'] hessi_inv= [[ 0

### optimized computational impl.

In [7]:
from functools import reduce
from tensorflow.python.ops import gradients_impl
from tensorflow.python.ops import array_ops, tensor_array_ops, control_flow_ops

def diagonal_hessians_highrank(ys, xs, gradients=None, name="hessians", colocate_gradients_with_ops=False,
            gate_gradients=False, aggregation_method=None):
  """Constructs the Hessian (one or more rank matrix) of sum of `ys` with respect to `x` in `xs`.
  `hessians_highrank()` adds ops to the graph to output the Hessian matrix of `ys`
  with respect to `xs`.  It returns a list of `Tensor` of length `len(xs)`
  where each tensor is the Hessian of `sum(ys)`. This function currently
  only supports evaluating the Hessian with respect to (a list of) one-
  dimensional tensors.
  The Hessian is a matrix of second-order partial derivatives of a scalar
  tensor (see https://en.wikipedia.org/wiki/Hessian_matrix for more details).
  Args:
    ys: A `Tensor` or list of tensors to be differentiated.
    xs: A `Tensor` or list of tensors to be used for differentiation.
    name: Optional name to use for grouping all the gradient ops together.
      defaults to 'hessians'.
    colocate_gradients_with_ops: See `gradients()` documentation for details.
    gate_gradients: See `gradients()` documentation for details.
    aggregation_method: See `gradients()` documentation for details.
  Returns:
    A list of Hessian matrices of `sum(ys)` for each `x` in `xs`.
  Raises:
    LookupError: if one of the operations between `xs` and `ys` does not
      have a registered gradient function.
  """
  xs = gradients_impl._AsList(xs)
  kwargs = {
    'colocate_gradients_with_ops': colocate_gradients_with_ops,
    'gate_gradients': gate_gradients,
    'aggregation_method': aggregation_method
  }
  # Compute first-order derivatives and iterate for each x in xs.
  hessians = []
  _gradients = tf.gradients(ys, xs, **kwargs) if gradients is None else gradients
  for i, _gradient, x in zip(range(len(xs)), _gradients, xs):
    shape = x.shape
    _gradient = tf.reshape(_gradient, [-1])
    
    n = tf.size(x)
    g = tf.gradients(_gradient*tf.eye(n), x)[0]
    hessian = tf.diag( tf.reshape(g, [-1]) )
    hessians.append(hessian)
  return hessians

In [8]:
sess = tf.InteractiveSession()

vs = tf.Variable(initial)
f = vs[0]**2 + 2.0*vs[0]*vs[1] + 3.0*vs[1]**2 + 4.0*vs[0] + 5.0*vs[1] + 6.0
sess.run(tf.global_variables_initializer())

for e in range(10):
    print('EPOCH', e, 'f=%.3f'%f.eval(), 'variables=', ['%.3f'%v for v in vs.eval()])
    
    grads = tf.gradients(f, vs, stop_gradients=vs)[0]
    hessi = diagonal_hessians_highrank(f, vs)[0]
    
    hessi_inv = tf.matrix_inverse(hessi)
    delta = - tf.reshape( tf.matmul(hessi_inv, tf.reshape(grads, [-1, 1])), [2] )
    print('\t \tgradients=', ['%.3f'%g for g in grads.eval()],
          'hessi_inv=', ('%s'%hessi_inv.eval()).replace('   ', '').replace('\n', ','),
          'delta=', ['%.3f'%d for d in delta.eval()])

    op = vs.assign( vs + delta )
    op.eval()

print('FINAL  ', 'f=%.3f'%f.eval(), 'variables=', ['%.3f'%v for v in vs.eval()])
sess.close()

EPOCH 0 f=9.758 variables= ['0.364', '0.324']
	 	gradients= ['5.376', '7.671'] hessi_inv= [[ 0.250.], [ 0.  0.125]] delta= ['-1.344', '-0.959']
EPOCH 1 f=2.320 variables= ['-0.980', '-0.635']
	 	gradients= ['0.770', '-0.770'] hessi_inv= [[ 0.250.], [ 0.  0.125]] delta= ['-0.192', '0.096']
EPOCH 2 f=2.125 variables= ['-1.173', '-0.539']
	 	gradients= ['0.577', '-0.577'] hessi_inv= [[ 0.250.], [ 0.  0.125]] delta= ['-0.144', '0.072']
EPOCH 3 f=2.016 variables= ['-1.317', '-0.467']
	 	gradients= ['0.433', '-0.433'] hessi_inv= [[ 0.250.], [ 0.  0.125]] delta= ['-0.108', '0.054']
EPOCH 4 f=1.954 variables= ['-1.425', '-0.412']
	 	gradients= ['0.325', '-0.325'] hessi_inv= [[ 0.250.], [ 0.  0.125]] delta= ['-0.081', '0.041']
EPOCH 5 f=1.920 variables= ['-1.506', '-0.372']
	 	gradients= ['0.244', '-0.244'] hessi_inv= [[ 0.250.], [ 0.  0.125]] delta= ['-0.061', '0.030']
EPOCH 6 f=1.900 variables= ['-1.567', '-0.341']
	 	gradients= ['0.183', '-0.183'] hessi_inv= [[ 0.250.], [ 0.  0.125]] delta= 